In [ ]:
# import time
# import json
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from pyproj import Proj, Transformer

# # -------------------- 기본 설정 --------------------
# driver = webdriver.Chrome()
# df = pd.DataFrame(columns=['동', 'utm_x', 'utm_y', '타입', '가격', '면적', '층'])
# dong = "예시동"

# # 거리 및 좌표 변환 설정
# px = driver.find_elements(By.CSS_SELECTOR, "div[style*='position: relative'][style*='width: 51px'][style*='height: 14px']")

# for div in px:
#     # width 추출
#     style = div.get_attribute("style")
#     width_value = None
#     if "width" in style:
#         for s in style.split(";"):
#             if "width" in s:
#                 width_value = s.strip().split(":")[1].strip()
#                 width_value = width_value[:width_value.find("px")]
#                 break

#     # 2. 해당 div 안의 span 찾기
#     try:
#         span = div.find_element(By.CSS_SELECTOR, "span[style*='display: block'][style*='font-size: 10px']")
#         span_text = span.text
#         span_text = span_text[:span_text.find("m")]
#     except:
#         span_text = None

# m, px = int(span_text), int(width_value)
# meter_per_px = m / px

# zone_number = 52
# northern = True

# # 좌표 변환기 설정
# utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
# wgs84_proj = Proj(proj='latlong', datum='WGS84')

# # 양방향 변환기
# transformer_to_wgs = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)
# transformer_to_utm = Transformer.from_proj(wgs84_proj, utm_proj, always_xy=True)
# transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)


# # -------------------- 좌표 로드 --------------------
# latlon_df = pd.read_csv('./asd.csv', header=None, names=['lat', 'lon'])

# for idx, row in latlon_df.iloc[150:151].iterrows():
#     lat, lon = row['lat'], row['lon']

#     # 위도/경도 → UTM 변환: 원점 설정
#     origin_x, origin_y = transformer_to_utm.transform(lon, lat)
#     print(f"원점 UTM 좌표 설정: {origin_x}, {origin_y}")

#     print(f"[{idx+1}] 이동: {lat}, {lon}")
#     url = f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
#     driver.get(url)
#     driver.maximize_window()
#     time.sleep(2)

#     # 핀 요소 찾기
#     try:
#         pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
#     except:
#         print("핀 로딩 실패")
#         continue

#     for i in range(len(pin_elements)):
#         while True:
#             try:
#                 map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
#                 my_x = map_element.size['width'] / 2 + driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']/2
#                 my_y = map_element.size['height'] / 2

#                 pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
#                 target_x = float(pin_elements[i].get_attribute("style").split(" ")[-1].replace("px;", ""))
#                 target_y = float(pin_elements[i].get_attribute("style").split(" ")[-3].replace("px;", ""))

#                 if abs(target_x) > 10000 or abs(target_y) > 10000:
#                     driver.refresh()
#                     time.sleep(3)
#                     continue
#                 else:
#                     break
#             except:
#                 time.sleep(1)

#         if target_x < 380 or target_y < -30:
#             continue

#         gap_x = (target_x - my_x) / 2
#         gap_y = (target_y - my_y) / 2

#         distance_x = gap_x * meter_per_px
#         distance_y = gap_y * meter_per_px

#         utm_x = origin_x - distance_x
#         utm_y = origin_y - distance_y

#         print(f"타깃 좌표 : x : {target_x}, y : {target_y}")
#         print("52 N", utm_x, utm_y)

#         try:
#             pin_elements[i].click()
#         except:
#             pass
#         time.sleep(1)

#         try:
#             body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
#             while True:
#                 body.send_keys(Keys.END)
#                 time.sleep(1)
#                 driver.find_element(By.CSS_SELECTOR, 'div.loader')
#         except:
#             pass

#         # ======================================================================================================================
#         items = driver.find_elements(By.CSS_SELECTOR, "div.item")
#         for i in range(len(items)):
#             items[i].click()
#             item2 = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
#             data = {}
#             stop_flag = False

#             items2 = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
#             for item in items2:
#                 if stop_flag:
#                     break

#                 ths = item.find_elements(By.CSS_SELECTOR, "th")
#                 tds = item.find_elements(By.CSS_SELECTOR, "td")
                
#                 if not ths or not tds:
#                     continue

#                 for th, td in zip(ths, tds):
#                     th_text = th.text.strip()
#                     td_text = td.text.strip()

#                     if "매물설명" in th_text:
#                         stop_flag = True
#                         break

#                     data[th_text] = td_text

#                 df.loc[len(df)] = [
#                     dong,
#                     utm_x,
#                     utm_y,
#                     data
#                 ]

#         # ======================================================================================================================


#         try:
#             pin_elements[i].click()
#         except:
#             pass
        
#         time.sleep(2)

# # -------------------- UTM → 위도/경도 변환 --------------------
# utm_coords = list(zip(df['utm_x'], df['utm_y']))
# latlon_list = [transformer.transform(x, y) for x, y in utm_coords]

# df['경도'] = [round(lon, 7) for lon, lat in latlon_list]
# df['위도'] = [round(lat, 7) for lon, lat in latlon_list]

# # -------------------- CSV 저장 --------------------
# df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
# df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)

# print("CSV 저장 완료")


In [ ]:
#  매물번호, 월세, m2, 층수, 추천업종, 매물특징 가져오기


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyproj import Proj, Transformer

# -------------------- 기본 설정 --------------------
driver = webdriver.Chrome()
driver.maximize_window()
dong = "예시동"
df = pd.DataFrame(columns=['동', 'utm_x', 'utm_y', '타입', '가격', '면적', '층'])

zone_number = 52
northern = True

# 좌표 변환기 설정
utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer_to_utm = Transformer.from_proj(wgs84_proj, utm_proj, always_xy=True)
transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)


# -------------------- meter_per_px 계산 함수 --------------------
def get_meter_per_px():
    driver.get("https://new.land.naver.com/offices?ms=37.5665,126.9780,18")  # 임시 위치
    time.sleep(2)

    px_divs = driver.find_elements(By.CSS_SELECTOR, "div[style*='position: relative'][style*='width: 51px'][style*='height: 14px']")
    for div in px_divs:
        try:
            style = div.get_attribute("style")
            width_value = int([s for s in style.split(";") if "width" in s][0].split(":")[1].replace("px", "").strip())
            span = div.find_element(By.CSS_SELECTOR, "span[style*='display: block'][style*='font-size: 10px']")
            meter_value = int(span.text.strip().replace("m", ""))
            return meter_value / width_value
        except:
            continue
    return 0.5  # fallback

meter_per_px = get_meter_per_px()


# -------------------- 좌표 로드 --------------------
latlon_df = pd.read_csv('../files/seogu_filt.csv', header=None, names=['lat', 'lon'])

for idx, row in latlon_df.iloc[104:105].iterrows():
    lat, lon = row['lat'], row['lon']
    origin_x, origin_y = transformer_to_utm.transform(lon, lat)

    print(f"[{idx+1}] 이동: {lat}, {lon}")
    driver.get(f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL")
    time.sleep(2)

    try:
        pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
    except:
        print("핀 로딩 실패")
        continue

    for i, pin in enumerate(pin_elements):
        try:
            map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
            list_width = driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']
            my_x = map_element.size['width'] / 2 + list_width / 2
            my_y = map_element.size['height'] / 2

            target_style = pin.get_attribute("style")
            target_x = float(target_style.split("left:")[1].split("px")[0].strip())
            target_y = float(target_style.split("top:")[1].split("px")[0].strip())

            # 좌표가 비정상적으로 튀는 경우 새로고침
            if abs(target_x) > 10000 or abs(target_y) > 10000:
                print(f"[좌표 이상] target_x: {target_x}, target_y: {target_y} → 새로고침")
                driver.refresh()
                time.sleep(3)
                continue

            # 기존 조건도 유지
            if target_x < 380 or target_y < -30:
                print(f"[건너뜀] target_x: {target_x}, target_y: {target_y}")
                continue


            distance_x = (target_x - my_x) / 2 * meter_per_px
            distance_y = (target_y - my_y) / 2 * meter_per_px

            utm_x = origin_x - distance_x
            utm_y = origin_y - distance_y

            print(f"  ↳ 핀 {i+1} → UTM: {utm_x}, {utm_y}")

            pin.click()
            time.sleep(1)

            try:
                body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
                while True:
                    body.send_keys(Keys.END)
                    time.sleep(1)
                    driver.find_element(By.CSS_SELECTOR, 'div.loader')  # 로딩 대기
            except:
                pass

            items = driver.find_elements(By.CSS_SELECTOR, "div.item")
            for item in items:
                try:
                    item.click()
                    time.sleep(0.5)
                    rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")

                    data = {"타입": "", "가격": "", "면적": "", "층": ""}
                    stop_flag = False

                    for row in rows:
                        if stop_flag:
                            break
                        ths = row.find_elements(By.CSS_SELECTOR, "th")
                        tds = row.find_elements(By.CSS_SELECTOR, "td")

                        for th, td in zip(ths, tds):
                            key, value = th.text.strip(), td.text.strip()
                            if "매물설명" in key:
                                stop_flag = True
                                break
                            if "타입" in key:
                                data["타입"] = value
                            elif "가격" in key:
                                data["가격"] = value
                            elif "면적" in key:
                                data["면적"] = value
                            elif "층" in key:
                                data["층"] = value

                    df.loc[len(df)] = [dong, utm_x, utm_y, data["타입"], data["가격"], data["면적"], data["층"]]

                except Exception as e:
                    print("데이터 수집 오류:", e)
                    continue

        except Exception as e:
            print(f"[클릭 실패] 좌표: x={target_x}, y={target_y} / 오류: {e}")
            continue


# -------------------- UTM → 위경도 변환 --------------------
utm_coords = list(zip(df['utm_x'], df['utm_y']))
latlon_list = [transformer.transform(x, y) for x, y in utm_coords]

df['경도'] = [round(lon, 7) for lon, lat in latlon_list]
df['위도'] = [round(lat, 7) for lon, lat in latlon_list]

# -------------------- CSV 저장 --------------------
df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)

print("CSV 저장 완료")


In [39]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyproj import Proj, Transformer


In [40]:
driver = webdriver.Chrome()
driver.maximize_window()
dong = "예시동"
df = pd.DataFrame(columns=['동', 'utm_x', 'utm_y', '타입', '가격', '면적', '층'])

zone_number = 52
northern = True

# 좌표 변환기 설정
utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer_to_utm = Transformer.from_proj(wgs84_proj, utm_proj, always_xy=True)
transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)

In [41]:
meter_per_px = 30 / 63
latlon_df = pd.read_csv('./files/seogu_filt.csv', header=None, names=['lat', 'lon'])

In [42]:
for idx, row in latlon_df.iloc[104:105].iterrows():
    lat, lon = row['lat'], row['lon']
    origin_x, origin_y = transformer_to_utm.transform(lon, lat)

    print(f"[{idx+1}] 이동: {lat}, {lon}")
    driver.get(f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL")
    time.sleep(2)

    try:
        pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
    except:
        print("핀 로딩 실패")
        continue


[105] 이동: 35.1389426, 126.8604109


In [45]:

try:
    body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
    while True:
        body.send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, 'div.loader')  # 로딩 대기
except:
    pass

items = driver.find_elements(By.CSS_SELECTOR, "div.item")
for item in items:
    try:
        item.click()
        time.sleep(0.5)
        rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")

        data = {"타입": "", "가격": "", "면적": "", "층": ""}
        stop_flag = False

        for row in rows:
            if stop_flag:
                break
            ths = row.find_elements(By.CSS_SELECTOR, "th")
            tds = row.find_elements(By.CSS_SELECTOR, "td")

            for th, td in zip(ths, tds):
                key, value = th.text.strip(), td.text.strip()
                if "매물설명" in key:
                    stop_flag = True
                    break
                if "타입" in key:
                    data["타입"] = value
                elif "가격" in key:
                    data["가격"] = value
                elif "면적" in key:
                    data["면적"] = value
                elif "층" in key:
                    data["층"] = value

        # df.loc[len(df)] = [dong, utm_x, utm_y, data["타입"], data["가격"], data["면적"], data["층"]]
        df.loc[len(df)] = [dong, data["타입"], data["가격"], data["면적"], data["층"]]

    except Exception as e:
        print("데이터 수집 오류:", e)
        continue


In [44]:
df = pd.DataFrame(columns=['동', '타입', '가격', '면적', '층'])


In [46]:
df

,동,타입,가격,면적,층
0,예시동,,,178.7㎡/135.55㎡(전용률76%),1/4층
1,예시동,,,172.47㎡/172.47㎡(전용률100%),2/6층
2,예시동,,,117㎡/117㎡(전용률100%),1/4층
3,예시동,,,105㎡/105㎡(전용률100%),1/4층
4,예시동,,,178.38㎡/178.38㎡(전용률100%),2/5층
5,예시동,,,178.38㎡/178.38㎡(전용률100%),2/5층
6,예시동,,,47.22㎡/47.22㎡(전용률100%),1/3층
7,예시동,,,47.22㎡/47.22㎡(전용률100%),1/3층
8,예시동,,,53.46㎡/53.46㎡(전용률100%),1/3층
9,예시동,,,53.46㎡/53.46㎡(전용률100%),1/3층


In [ ]:
rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
data = {"타입": "", "가격": "", "면적": "", "층": ""}
stop_flag = False

for row in rows:
    if stop_flag:
        break
    ths = row.find_elements(By.CSS_SELECTOR, "th")
    tds = row.find_elements(By.CSS_SELECTOR, "td")

    for th, td in zip(ths, tds):
        key, value = th.text.strip(), td.text.strip()
        if "매물설명" in key:
            stop_flag = True
            break
        if "타입" in key:
            data["타입"] = value
        elif "가격" in key:
            data["가격"] = value
        elif "면적" in key:
            data["면적"] = value
        elif "층" in key:
            data["층"] = value

[<selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db9960ccb0b7c3809c3cc", element="f.A965A66217B65F267A432C5D71D17B66.d.11DCE975D2AC85D7771ECC7FAE907E96.e.1485")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db9960ccb0b7c3809c3cc", element="f.A965A66217B65F267A432C5D71D17B66.d.11DCE975D2AC85D7771ECC7FAE907E96.e.1486")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db9960ccb0b7c3809c3cc", element="f.A965A66217B65F267A432C5D71D17B66.d.11DCE975D2AC85D7771ECC7FAE907E96.e.1487")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db9960ccb0b7c3809c3cc", element="f.A965A66217B65F267A432C5D71D17B66.d.11DCE975D2AC85D7771ECC7FAE907E96.e.1488")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db9960ccb0b7c3809c3cc", element="f.A965A66217B65F267A432C5D71D17B66.d.11DCE975D2AC85D7771ECC7FAE907E96.e.1489")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dc4c06341f3db99